<a href="https://colab.research.google.com/github/sda96/AIFFEL_3rd_hackerton_TUNiB_DKTC/blob/main/notebook/SeungTaek/gpt2_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following packages were automatically installed and are no longer required:
  cuda-command-line-tools-10-0 cuda-command-line-tools-10-1
  cuda-command-line-tools-11-0 cuda-compiler-10-0 cuda-compiler-10-1
  cuda-compiler-11-0 cuda-cuobjdump-10-0 cuda-cuobjdump-10-1
  cuda-cuobjdump-11-0 cuda-cupti-10-0 cuda-cupti-10-1 cuda-cupti-11-0
  cuda-cupti-dev-11-0 cuda-documentation-10-0 cuda-documentation-10-1
  cuda-documentation-11-0 cuda-documentation-11-1 cuda-gdb-10-0 cuda-gdb-10-1
  cuda-gdb-11-0 cuda-gpu-library-advisor-10-0 cuda-gpu-library-advisor-10-1
  cuda-libraries-10-0 cuda-libraries-10-1 cuda-libraries-11-0
  cuda-memcheck-10-0 cuda-memcheck-10-1 cuda-memcheck-11-0 cuda-nsight-10-0
  cuda-nsight-10-1 cuda-nsight-11-0 cuda-nsight-11-1 cuda-nsight-compute-10-0
  cuda-nsight-compute-10-1 cuda-nsight-compute-11-0 cuda-nsight-comp

In [2]:
!gdown https://drive.google.com/uc?id=1A8IdrqgEjRhvg9KoV6dopo5GirTKO3aj
!gdown https://drive.google.com/uc?id=1SCxtUqlGEzYCfCbHUREFf5UqoM1uWDBK
!gdown https://drive.google.com/uc?id=1Zh4mCpgQReE4E0sAjiFXwSV6Y7lc77fO
!gdown https://drive.google.com/uc?id=19jT2ZTb6oj6sFERkl5AT3qQExxN3D5cK

Downloading...
From: https://drive.google.com/uc?id=1A8IdrqgEjRhvg9KoV6dopo5GirTKO3aj
To: /content/sns_val.zip
100% 34.8M/34.8M [00:00<00:00, 303MB/s]
Downloading...
From: https://drive.google.com/uc?id=1SCxtUqlGEzYCfCbHUREFf5UqoM1uWDBK
To: /content/sns_train.zip
100% 278M/278M [00:00<00:00, 389MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Zh4mCpgQReE4E0sAjiFXwSV6Y7lc77fO
To: /content/train.csv
100% 2.24M/2.24M [00:00<00:00, 176MB/s]
Downloading...
From: https://drive.google.com/uc?id=19jT2ZTb6oj6sFERkl5AT3qQExxN3D5cK
To: /content/test.json
100% 417k/417k [00:00<00:00, 106MB/s]


In [3]:
!unzip -uq "/content/sns_train.zip" -d "/content/sns_data"
!unzip -uq "/content/sns_val.zip" -d "/content/sns_data"

In [4]:
!pip install transformers

In [5]:
#!pip install tensorflow==2.5.0

In [6]:
import pandas as pd # pandas 모듈 로드
import json  # json 모듈 로드
import re
import os
from glob import glob
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from collections import defaultdict, Counter

import tensorflow as tf
from transformers import TextClassificationPipeline
from transformers import BertTokenizer, TFBertForSequenceClassification   #BERT
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification   #RoBerta
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import TFGPT2LMHeadModel, TFGPT2ForSequenceClassification, TFGPT2Model    #GPT2

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [7]:
train_df = pd.read_csv("/content/train.csv")

In [8]:
train_df

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
3945,3945,기타 괴롭힘 대화,준하야 넌 대가리가 왜이렇게 크냐?\n내 머리가 뭐.\n밥먹으면 대가리만 크냐 너는...
3946,3946,갈취 대화,내가 지금 너 아들 김길준 데리고 있어. 살리고 싶으면 계좌에 1억만 보내\n예.?...
3947,3947,직장 내 괴롭힘 대화,나는 씨 같은 사람 보면 참 신기하더라. 어떻게 저렇게 살지.\n왜 그래. 들리겠어...
3948,3948,갈취 대화,누구맘대로 여기서 장사하래?\n이게 무슨일입니까?\n남의 구역에서 장사하려면 자릿세...


In [9]:
data_path ="/content/sns_data/content/sns_csv_train"
files = glob(data_path + "/*")
files

['/content/sns_data/content/sns_csv_train/여가생활.csv',
 '/content/sns_data/content/sns_csv_train/행사.csv',
 '/content/sns_data/content/sns_csv_train/주거와생활.csv',
 '/content/sns_data/content/sns_csv_train/일과직업.csv',
 '/content/sns_data/content/sns_csv_train/식음료.csv',
 '/content/sns_data/content/sns_csv_train/미용과건강.csv',
 '/content/sns_data/content/sns_csv_train/개인및관계.csv',
 '/content/sns_data/content/sns_csv_train/시사교육.csv',
 '/content/sns_data/content/sns_csv_train/상거래(쇼핑).csv']

In [10]:
df_all = pd.DataFrame()
for i in files:
    df = pd.read_csv(i)
    df_all = pd.concat([df_all, df])

In [11]:
df_all

,0
0,더쿠발 펜하스포 주웠는데 보내줄까?!\n안돼야 ㅋㅋㅋㅋㅋ\n나 더큐에서본거같은디 스...
1,이건 내것인디 이거뚜 조오ㅓ 저것두 얇긴얇윰\n헐랭 이거는 짱 싸네 조은정보 감사요...
2,나 인크레더블 보러옴 ㅋ\n초딩들 우르르륵\n존잼이야 제발봐 ㅠㅠ\n얼마나 재밌길래...
3,#@이름# #@이름#한테 위키미키 아냐고 물어봐 ㅎ\n위키미키는 나도알아 ㅋㅋㅋ #...
4,밥먹으면서 힐하우스보는중..... #@이름#는 밥먹구있냐...ㅠㅠ\n웅웅 ㅋㅋㅋㅋㅋ...
...,...
115480,이제은행가볼라고\nㅋㅋㅋ결정의 시간이군 엘피판 이문세꺼 하나 더 사는거 어때? ㅋㅋ...
115481,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ #@URL# 이거 마자?\n응 근데 이것도 배송비 다 따로 일...
115482,근데이새끼는 내가저점에서삿는데도 또떨어지노\nㅇㅇ 내말이 ㅜ #@기타#~\n바닥이 ...
115483,먼가 티가안나 벽에 붙인거봣는데 그래 초록별도해\n뭐져\n붙여\nㅋㅋㅋㅋㅋ 초록별 ...


In [12]:
df_all["class"] = "일반 대화"

In [13]:
df_all.head()

,0,class
0,더쿠발 펜하스포 주웠는데 보내줄까?!\n안돼야 ㅋㅋㅋㅋㅋ\n나 더큐에서본거같은디 스...,일반 대화
1,이건 내것인디 이거뚜 조오ㅓ 저것두 얇긴얇윰\n헐랭 이거는 짱 싸네 조은정보 감사요...,일반 대화
2,나 인크레더블 보러옴 ㅋ\n초딩들 우르르륵\n존잼이야 제발봐 ㅠㅠ\n얼마나 재밌길래...,일반 대화
3,#@이름# #@이름#한테 위키미키 아냐고 물어봐 ㅎ\n위키미키는 나도알아 ㅋㅋㅋ #...,일반 대화
4,밥먹으면서 힐하우스보는중..... #@이름#는 밥먹구있냐...ㅠㅠ\n웅웅 ㅋㅋㅋㅋㅋ...,일반 대화


In [14]:
df_all = df_all.rename(columns = {'0' : "conversation"})

In [15]:
df_all

,conversation,class
0,더쿠발 펜하스포 주웠는데 보내줄까?!\n안돼야 ㅋㅋㅋㅋㅋ\n나 더큐에서본거같은디 스...,일반 대화
1,이건 내것인디 이거뚜 조오ㅓ 저것두 얇긴얇윰\n헐랭 이거는 짱 싸네 조은정보 감사요...,일반 대화
2,나 인크레더블 보러옴 ㅋ\n초딩들 우르르륵\n존잼이야 제발봐 ㅠㅠ\n얼마나 재밌길래...,일반 대화
3,#@이름# #@이름#한테 위키미키 아냐고 물어봐 ㅎ\n위키미키는 나도알아 ㅋㅋㅋ #...,일반 대화
4,밥먹으면서 힐하우스보는중..... #@이름#는 밥먹구있냐...ㅠㅠ\n웅웅 ㅋㅋㅋㅋㅋ...,일반 대화
...,...,...
115480,이제은행가볼라고\nㅋㅋㅋ결정의 시간이군 엘피판 이문세꺼 하나 더 사는거 어때? ㅋㅋ...,일반 대화
115481,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ #@URL# 이거 마자?\n응 근데 이것도 배송비 다 따로 일...,일반 대화
115482,근데이새끼는 내가저점에서삿는데도 또떨어지노\nㅇㅇ 내말이 ㅜ #@기타#~\n바닥이 ...,일반 대화
115483,먼가 티가안나 벽에 붙인거봣는데 그래 초록별도해\n뭐져\n붙여\nㅋㅋㅋㅋㅋ 초록별 ...,일반 대화


In [16]:
df_all = df_all[['class', 'conversation']]
df_all

,class,conversation
0,일반 대화,더쿠발 펜하스포 주웠는데 보내줄까?!\n안돼야 ㅋㅋㅋㅋㅋ\n나 더큐에서본거같은디 스...
1,일반 대화,이건 내것인디 이거뚜 조오ㅓ 저것두 얇긴얇윰\n헐랭 이거는 짱 싸네 조은정보 감사요...
2,일반 대화,나 인크레더블 보러옴 ㅋ\n초딩들 우르르륵\n존잼이야 제발봐 ㅠㅠ\n얼마나 재밌길래...
3,일반 대화,#@이름# #@이름#한테 위키미키 아냐고 물어봐 ㅎ\n위키미키는 나도알아 ㅋㅋㅋ #...
4,일반 대화,밥먹으면서 힐하우스보는중..... #@이름#는 밥먹구있냐...ㅠㅠ\n웅웅 ㅋㅋㅋㅋㅋ...
...,...,...
115480,일반 대화,이제은행가볼라고\nㅋㅋㅋ결정의 시간이군 엘피판 이문세꺼 하나 더 사는거 어때? ㅋㅋ...
115481,일반 대화,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ #@URL# 이거 마자?\n응 근데 이것도 배송비 다 따로 일...
115482,일반 대화,근데이새끼는 내가저점에서삿는데도 또떨어지노\nㅇㅇ 내말이 ㅜ #@기타#~\n바닥이 ...
115483,일반 대화,먼가 티가안나 벽에 붙인거봣는데 그래 초록별도해\n뭐져\n붙여\nㅋㅋㅋㅋㅋ 초록별 ...


In [17]:
df_train2 = df_all.sample(n = 900, random_state = 5)

In [18]:
df_train = pd.concat([train_df, df_train2])
df_train

,idx,class,conversation
0,0.0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1.0,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2.0,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3.0,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4.0,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...,...
68988,NaN,일반 대화,라는 단어를\n국립대학육성사업과 관하여 성과관리 및 평가 내용 등의 공유를 위한 환...
187001,NaN,일반 대화,어디냐\n도서관쓰요\n차 가져옴?ㅎㅎ 언제가~?\n아직도서관이얔ㅋㅋㅋㅋㅋㅋ너아직학교...
64059,NaN,일반 대화,ㅋㅋㅋㅋ공머티콘 사고싶어 어쩔거야 마지막 에프가 내미래같아섴ㅋㅋㅋㅋ동질감 느끼는중\...
10830,NaN,일반 대화,흠 #@이모티콘#\n신뢰면 아닌가 영원과 사랑이 다이아몬든가 ㅋㅋ 왜?\n오빠가 날...


In [19]:
df_train = df_train.drop(['idx'],axis=1)
df_train

,class,conversation
0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...
...,...,...
68988,일반 대화,라는 단어를\n국립대학육성사업과 관하여 성과관리 및 평가 내용 등의 공유를 위한 환...
187001,일반 대화,어디냐\n도서관쓰요\n차 가져옴?ㅎㅎ 언제가~?\n아직도서관이얔ㅋㅋㅋㅋㅋㅋ너아직학교...
64059,일반 대화,ㅋㅋㅋㅋ공머티콘 사고싶어 어쩔거야 마지막 에프가 내미래같아섴ㅋㅋㅋㅋ동질감 느끼는중\...
10830,일반 대화,흠 #@이모티콘#\n신뢰면 아닌가 영원과 사랑이 다이아몬든가 ㅋㅋ 왜?\n오빠가 날...


In [20]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train

,class,conversation
0,기타 괴롭힘 대화,당신 어디부서야 사장나오라그래\n고객님 죄송합니다. 이건 제 선에서는.처리가 안되는...
1,직장 내 괴롭힘 대화,자네 뭐하나?\n아 지금 오후 회의자료 만드는 중입니다\n아그래? 그거 다하면 뭐 ...
2,직장 내 괴롭힘 대화,선영씨 오늘 저녁에 약속있어요?\n 아니요. 없습니다.\n잘됐네. 그럼 남아서 회의...
3,갈취 대화,지수야 나 부모님 생신 선물 사는데 돈 좀 빌려주라 .\n미안해 . 나도 돈이 없어...
4,직장 내 괴롭힘 대화,우리 성근씨는 왜 다른 팀원들이랑 잘 융화가 안되는 거 같지?\n아.네 제가 성격이...
...,...,...
4845,협박 대화,우리 헤어졌는데 왜 이렇게 찾아와요.\n난 못 헤어진다고 했잖아. 도대체 왜 그러는...
4846,갈취 대화,이 새끼 이거 봐라 찌질한 새끼. 거지새끼가 어디서 돈이 나서 이렇게 좋은 옷을 입...
4847,일반 대화,지금 렙6인데 ..\n왓 크리스탈 다 받을람 100레벨은 되야 하는데 오늘안에 오르...
4848,기타 괴롭힘 대화,야 너 옷 뭐냐. 원피스 이쁘네.\n어어 이번에 새로 샀어. 고마워.\n근데 색깔이...


In [21]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣0-9?.!,\n]+", " ", sentence)
    sentence = re.sub('\n'," "+"\n"+" ",sentence)

    sentence = sentence.strip()

    return sentence

In [22]:
df_train["conversation"] = df_train["conversation"].apply(preprocess_sentence)
df_train

,class,conversation
0,기타 괴롭힘 대화,당신 어디부서야 사장나오라그래 \n 고객님 죄송합니다 . 이건 제 선에서는 . 처리...
1,직장 내 괴롭힘 대화,자네 뭐하나 ? \n 아 지금 오후 회의자료 만드는 중입니다 \n 아그래 ? 그거...
2,직장 내 괴롭힘 대화,선영씨 오늘 저녁에 약속있어요 ? \n 아니요 . 없습니다 . \n 잘됐네 ....
3,갈취 대화,지수야 나 부모님 생신 선물 사는데 돈 좀 빌려주라 . \n 미안해 . 나도 돈이...
4,직장 내 괴롭힘 대화,우리 성근씨는 왜 다른 팀원들이랑 잘 융화가 안되는 거 같지 ? \n 아 . 네 ...
...,...,...
4845,협박 대화,우리 헤어졌는데 왜 이렇게 찾아와요 . \n 난 못 헤어진다고 했잖아 . 도대체 ...
4846,갈취 대화,이 새끼 이거 봐라 찌질한 새끼 . 거지새끼가 어디서 돈이 나서 이렇게 좋은 옷을 ...
4847,일반 대화,지금 렙6인데 . . \n 왓 크리스탈 다 받을람 100레벨은 되야 하는데 오늘안...
4848,기타 괴롭힘 대화,야 너 옷 뭐냐 . 원피스 이쁘네 . \n 어어 이번에 새로 샀어 . 고마워 . ...


In [23]:
bucket_class = []
label_names = ["협박 대화", "갈취 대화", "직장 내 괴롭힘 대화", 
               "기타 괴롭힘 대화", "일반 대화"]

for idx, label in enumerate(df_train["class"]):
  if label == label_names[0]:
    bucket_class.append(0)
  elif label == label_names[1]:
    bucket_class.append(1)
  elif label == label_names[2]:
    bucket_class.append(2)
  elif label == label_names[3]:
    bucket_class.append(3)
  elif label == label_names[4]:
    bucket_class.append(4)

df_train["class"] = bucket_class
df_train.head()

,class,conversation
0,3,당신 어디부서야 사장나오라그래 \n 고객님 죄송합니다 . 이건 제 선에서는 . 처리...
1,2,자네 뭐하나 ? \n 아 지금 오후 회의자료 만드는 중입니다 \n 아그래 ? 그거...
2,2,선영씨 오늘 저녁에 약속있어요 ? \n 아니요 . 없습니다 . \n 잘됐네 ....
3,1,지수야 나 부모님 생신 선물 사는데 돈 좀 빌려주라 . \n 미안해 . 나도 돈이...
4,2,우리 성근씨는 왜 다른 팀원들이랑 잘 융화가 안되는 거 같지 ? \n 아 . 네 ...


In [24]:
train_df, test_df = train_test_split(df_train, test_size = 0.2)

In [25]:
test_df

,class,conversation
4499,1,여보 이거는 우리애 돌반지잖아요 \n 그게왜 ? ? \n 이거는 가져가지 말아요 ...
3471,3,내 레포트 내일까지 써 와 . \n 내가 왜 ? \n 내 말 안들으면 니 나체 ...
2516,3,너 언제부터 휠체어 타고 다녔냐 ? \n 나 어려서부터 \n 그럼 니 부모님이 너...
4369,3,인테리어비 좀 깍아주세요 \n 네 ? 이것도 진짜 많이 깍아 드린건데요 \n 아 그...
3272,2,기민씨 이 파일 내일까지 완성해 . \n 네 ? 이건 적어도 삼일은 . \n 말...
...,...,...
4271,4,구래ㅎㅎㅎㅎ곧봐 ! 나 상인현대맨션 ? 거기 내릴게 ! 5번출구쪽 \n 웅 ! 언제...
4022,3,야 너 왜아빠가 없어 ? \n 어렸을때 이혼하셨대 \n 그래 ? ? 그럼 니네 엄...
1541,0,다 죽여버리기전에 여기에 돈 담아 ! \n 꺄악 . ! ! 도둑 . 도둑이야 . ...
2593,0,문 안잡아줘서 다친거라면 앞사람이 잘못한거아닌가요 ? \n 잉 ? 왜 태클 ? ?...


In [26]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', pad_token='<pad>')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [27]:
max_seq_len = 128

In [28]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer):

    input_ids, data_labels = [], []
    
    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        bos_token = [tokenizer.bos_token]
        eos_token = [tokenizer.eos_token]
        tokens = bos_token + tokenizer.tokenize(example) + eos_token
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        input_id = pad_sequences([input_id], maxlen=max_seq_len, value=tokenizer.pad_token_id, padding='post')[0]

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        input_ids.append(input_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return input_ids, data_labels

In [29]:
import pandas as pd
import numpy as np
import urllib.request
import os
from tqdm import tqdm
import tensorflow as tf
from sklearn import preprocessing
from transformers import AutoTokenizer, TFGPT2Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [30]:
train_X, train_y = convert_examples_to_features(train_df['conversation'], train_df['class'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 3880/3880 [00:02<00:00, 1910.52it/s]


In [80]:
test_X, test_y = convert_examples_to_features(test_df['conversation'], test_df['class'], max_seq_len=max_seq_len, tokenizer=tokenizer)

100%|██████████| 970/970 [00:00<00:00, 1786.46it/s]


In [32]:
model = TFGPT2Model.from_pretrained('skt/kogpt2-base-v2', from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [33]:
max_seq_len = 128

In [34]:
input_ids_layer = tf.keras.layers.Input(shape=(max_seq_len,), dtype=tf.int32)
outputs = model([input_ids_layer])

In [35]:
class TFGPT2ForSequenceClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFGPT2ForSequenceClassification, self).__init__()
        self.gpt = TFGPT2Model.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, inputs):
        outputs = self.gpt(input_ids=inputs)
        cls_token = outputs[0][:, -1]
        prediction = self.classifier(cls_token)

        return prediction

In [36]:
# TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.118.16.42:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.118.16.42:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [37]:
strategy = tf.distribute.experimental.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [38]:
with strategy.scope():
  model = TFGPT2ForSequenceClassification("skt/kogpt2-base-v2",
                                          num_labels = 5)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2Model: ['transformer.h.4.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2Model from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2Model from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All t

In [39]:
model.fit(train_X, train_y, epochs=2, batch_size=32, validation_split=0.2)

Epoch 1/2
Instructions for updating:
use `experimental_local_results` instead.


Instructions for updating:
use `experimental_local_results` instead.


97/97 [==============================] - 81s 197ms/step - loss: 0.9774 - accuracy: 0.6782 - val_loss: 0.4315 - val_accuracy: 0.8273
Epoch 2/2
97/97 [==============================] - 8s 80ms/step - loss: 0.3441 - accuracy: 0.8872 - val_loss: 0.4093 - val_accuracy: 0.8673


In [40]:
results = model.evaluate(test_X, test_y, batch_size=1024)
print("test loss, test acc: ", results)

1/1 [==============================] - 6s 6s/step - loss: 0.3678 - accuracy: 0.8639
test loss, test acc:  [0.36781781911849976, 0.8639175295829773]


In [44]:
model.summary()

Model: "tfgpt2_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tfgpt2_model_1 (TFGPT2Model  multiple                 125164032 
 )                                                               
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
Total params: 125,167,877
Trainable params: 125,167,877
Non-trainable params: 0
_________________________________________________________________


In [46]:
model_path = "./model/fine-tuned-gpt2"

In [47]:
tokenizer.save_pretrained(model_path)

('./model/fine-tuned-gpt2/tokenizer_config.json',
 './model/fine-tuned-gpt2/special_tokens_map.json',
 './model/fine-tuned-gpt2/vocab.json',
 './model/fine-tuned-gpt2/merges.txt',
 './model/fine-tuned-gpt2/added_tokens.json',
 './model/fine-tuned-gpt2/tokenizer.json')

In [101]:
model.save_pretrained(model_path)

AttributeError: ignored

In [74]:
loaded_tokenizer = AutoTokenizer.from_pretrained(model_path)

loaded_model = TFGPT2Model.from_pretrained(model_path)


text_classifier = TextClassificationPipeline(
    tokenizer=loaded_tokenizer, 
    model=loaded_model, 
    framework='tf',
    return_all_scores=True
)

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at ./model/fine-tuned-gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.
The model 'TFGPT2Model' is not supported for . Supported models are ['TFDebertaV2ForSequenceClassification', 'TFDebertaForSequenceClassification', 'TFRemBertForSequenceClassification', 'TFRoFormerForSequenceClassification', 'TFConvBertForSequenceClassification', 'TFDistilBertForSequenceClassification', 'TFAlbertForSequenceClassification', 'TFCamembertForSequenceClassification', 'TFXLMRobertaForSequenceClassification', 'TFLongformerForSequenceClassification', 'TFRobertaForSequenceClassification', 'TFLayoutLMForSequenceClassification', 'TFBertForSequenceClassification', 'TFXLNetForSequenceClassification', 'TFMobileBertForSequenceClassification', 'TFFlaubertForSequenceC

In [96]:
loaded_model.predict(test_X)

array([[ 9051,  9782,  9071, ...,   739,   376,     1],
       [    1,  9094, 10286, ...,     3,     3,     3],
       [    1, 10099, 40595, ...,     3,     3,     3],
       ...,
       [  406,   739,  9051, ...,   739,   376,     1],
       [ 9298,  8507,   739, ...,   739,   376,     1],
       [14701,  7991,   739, ...,  8244, 36510,     1]])

In [77]:
predicted_label_list = []
predicted_score_list = []

for text in tqdm(test_X):
    # predict
    preds_list = text_classifier(text)[0]


    sorted_preds_list = max(preds_list, key=lambda x: x['score'])
    predicted_label_list.append(sorted_preds_list["label"]) # label
    predicted_score_list.append(sorted_preds_list["score"]) # score

  0%|          | 0/970 [00:00<?, ?it/s]


KeyError: ignored

In [76]:
text_classifier(test_X[0])

KeyError: ignored